In [32]:
def lire(path):
    with open(path, encoding= 'utf-8') as input_stream: #***
        return [tokenizer.tokenize(sentence) for sentence in sent_tokenize(input_stream.read())] #***

In [33]:
corpus = [] #changez le chemin 
corpus.extend(lire("C:/Users/andre/OneDrive/Bureau/S1_2026_USN/machine_probing_2025/MARKOV/proust/2998-0.txt"))
corpus.extend(lire("C:/Users/andre/OneDrive/Bureau/S1_2026_USN/machine_probing_2025/MARKOV/proust/2999-0.txt"))
corpus.extend(lire("C:/Users/andre/OneDrive/Bureau/S1_2026_USN/machine_probing_2025/MARKOV/proust/3000-0.txt"))

# Exercice 1 : chaîne de Markov de premier ordre (unigrammes)

## a. Compter les transitions

Écrivez une fonction `compter_transitions_unigrammes` qui, étant donné un corpus (séquences de phrases, une phrase étant une séquence de mot), va compter les différentes transitions d'un mot à l'autre dans le corpus.

Le résultat prendra la forme d'un dictionnaire à deux niveaux :
- Le premier niveau sera le mot précédent
- le second niveau sera le comptage du mot courant étant donné le mot précédent.

In [10]:
def compter_transitions_unigrammes(corpus):
    transitions = {}

    for phrase in corpus:
        prev = ""
        for token in phrase:
            if prev not in transitions:
                transitions[prev] = {}
            if token not in transitions[prev]: #***
                transitions[prev][token] = 0   #***
            transitions[prev][token] += 1      #***
            prev = token                       #***

    return transitions

## b. Transformer des comptes en probabilité

Écrivez une fonction `probabilifier` qui, étant donné des comptes de transitions, transforme ces comptes en probabilité.

La somme des probabilités des transitions partant d'un même mot doit sommer à 1 (ou presque 1, les nombres étant petits, des imprécisions peuvent arriver).

Par exemple, la somme des transitions partant du mot `Si` doit sommer à 1.

In [11]:
def probabilifier(comptes_transitions):
    for token in comptes_transitions:
        total = sum(comptes_transitions[token].values()) #***
        for nxt in comptes_transitions[token]:           #***
            comptes_transitions[token][nxt] /= total     #***

    return comptes_transitions

## c. Créer une chaîne de Markov d'ordre 1

En utilisant les fonctions précédentes, écrivez une fonction `chaine_markov_unigramme` qui, étant donné un corpus, renvoie une chaîne de Markov d'ordre 1 (probabilités de transitions d'un mot à un autre).

In [12]:
def chaine_markov_unigramme(corpus):
    transitions = compter_transitions_unigrammes(corpus)

    return probabilifier(transitions)

In [13]:
markov_chain = chaine_markov_unigramme(corpus) #***

## d. Générer des phrases avec votre chaîne de Markov

Écrivez une fonction `generer_unigramme` qui, étant donné un token de départ, va générer une phrase.

On considère que nous sommes à la fin d'une phrase dès que l'on atteint une ponctuation forte ('.', '?', '!').

Pour générer le prochain mot, on prendra systématiquement le mot le plus probable étant donné le mot précédent.

Afin de vérifier le bon fonctionnement de la fonction, vous générerez d'abord une phrase commençant par `Si`.

In [36]:
def generer_unigramme(markov_chain, start_token):
    maximum = NB_MOTS_MAXI
    token = start_token
    print(token, end=" ")
    prev = token
    while token not in ponctuation and maximum > 0: #***
        nxt = sorted([x for x in markov_chain[prev].items()], key=lambda x: -x[1]) #***
        token = nxt[0][0] #***
        prev = token #***
        maximum -= 1 #***
        print(token, end=" ") #***

In [46]:
generer_unigramme(markov_chain, "Si") #*** modifiez la cellule ci-dessus pour n'afficher que 50 mots (là il y en 100 en output)
print (30*'*')
generer_unigramme(markov_chain, "Et") #*** expliquez pourquoi cela marche
print (30*'*')
generer_unigramme(markov_chain, "Luc") #*** expliquez pourquoi cela ne marche pas

Si vous avez pas , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et ******************************
Et comme un peu de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la vie , et de la ******************************


## e. Améliorer la génération

Comme vous avez pu le constater, la génération est assez... décevante. Pour améliorer la génération automatique, nous allons ajouter un peu d'aléatoire.

Modifiez la fonction `generer_unigramme` pour y ajouter un argument `n_best` qui dira de prendre non pas le mot le plus probable, mais un mot au hasard parmi les `n_best` plus probables.

In [47]:
def generer_unigramme(markov_chain, start_token, n_best=1):
    maximum = NB_MOTS_MAXI
    token = start_token
    print(token, end=" ")
    prev = token
    while token not in ponctuation and maximum > 0:
        nxt = sorted([x for x in markov_chain[prev].items()], key=lambda x: -x[1])
        token = random.choice(nxt[:n_best])[0] #***
        prev = token #***
        maximum -= 1 #***
        print(token, end=" ")

In [54]:
generer_unigramme(markov_chain, "Et", 2) #***

Et comme une fois que je n' était pas , mais qui n' était pas , et de la vie , mais qui n' est pas , et qui n' était plus de l' autre , mais qui n' était pas , et de l' autre , mais je ne me semblait pas , et de l' air d' une autre part , et de l' air d' une autre part , mais qui ne me semblait que je ne me dit que je ne me semblait pas , mais qui ne me semblait pas de la vie , et qui 

In [58]:
generer_unigramme(markov_chain, "Si", 10) #***

Si l' on a pu croire pendant une grande dame à Balbec pour les jeunes qui n' étaient pas que vous a eu de la plus en avait pas la même pas la plus d' être plus de l' avait eu pour le temps . 

# Exercice 2 : chaîne de Markov d'ordre 2 (bigrammes)

Nous avons fini par avoir une génération acceptable, mais elle demeure encore un peu incohérente. Afin d'améliorer la cohérence de la génération des mots, nous allons utiliser un contexte plus grand afin de générer les mots.

Reprenez les fonctions précédentes (à part `probabilifier` qui restera la même) afin d'utiliser non pas un mot, mais deux mots en contexte.

Si deux mots sont utilisés comme contexte, ils doivent être séparés par une espace.

In [59]:
def compter_transitions_bigrammes(corpus):
    transitions = {}

    for phrase in corpus:
        prevs = ["", ""] #***
        for token in phrase:
            prev = " ".join(prevs) #***
            if prev not in transitions: #***
                transitions[prev] = {} #***
            if token not in transitions[prev]: #***
                transitions[prev][token] = 0 #***
            transitions[prev][token] += 1 #***
            prevs[0] = prevs[1] #***
            prevs[1] = token #***

    return transitions

In [60]:
def chaine_markov_bigramme(corpus):
    transitions = compter_transitions_bigrammes(corpus)

    return probabilifier(transitions)

In [61]:
markov_chain = chaine_markov_bigramme(corpus)

In [63]:
def generate_bi(markov_chain, start_token):
    maximum = NB_MOTS_MAXI
    token = start_token
    print(token, end=" ")
    prevs = ["", ""]
    prevs[1] = token
    while token not in ponctuation and maximum > 0:
        prev = " ".join(prevs)
        nxt = sorted([x for x in markov_chain[prev].items()], key=lambda x: -x[1])
        token = nxt[0][0]
        prevs[0] = prevs[1]
        prevs[1] = token
        maximum -= 1
        print(token, end=" ")

In [67]:
generate_bi(markov_chain, "Si")

Si j' avais été sur la plage , nous ne sommes pas accoutumés , le jour où je ne pouvais pas venir , au contraire , l' idée de moi , mais qui ne sont pas les mêmes que font travailler pour elles , et qui , à la fois , et qui , à la fois , et qui , à la fois , et qui , à la fois , et qui , à la fois , et qui , à la fois , et qui , à la fois , et qui , à la fois , et 

In [68]:
def generate_bi(markov_chain, start_token, n_best=1):
    maximum = NB_MOTS_MAXI
    token = start_token
    print(token, end=" ")
    prevs = ["", ""]
    prevs[1] = token
    while token not in ponctuation and maximum > 0:
        prev = " ".join(prevs)
        nxt = sorted([x for x in markov_chain[prev].items()], key=lambda x: -x[1])
        token = random.choice(nxt[:n_best])[0]
        prevs[0] = prevs[1]
        prevs[1] = token
        maximum -= 1
        print(token, end=" ")

In [71]:
generate_bi(markov_chain, "Si", 2)

Si j' avais été voir des tempêtes que sur un ton d' indifférence qu' on ne le connaît pas , et que j' étais venu chercher dans ses livres , les jours où je n' avais jamais songé qu' il y a une de ces jeunes gens--on en verra qui étaient venus la voir . 

In [73]:
generate_bi(markov_chain, "Si", 10)

Si je l' eusse reprise , cette heure d' air et en Amérique , ou comme à une soirée musicale qui perdrait de son visage était plus particulièrement câline avec Swann et que la photographie préférée de Swann qu' il était bien entendu ) avec quelle malice il a prononcé à l' entendre parler des tableaux achetés on ne s' y embrouille dans ces cortèges , un homme qui a cherché de toutes , ni d' autre aux femmes , tâchent de les aggraver , plus d' importance pour moi à Gilberte en faisant semblant d' ignorer que vous , Odette 

# Exercice 3 : chaîne de Markov d'ordre arbitraire

Nous y sommes presque ! Nous commençons à avoir une génération de qualité acceptable.

Afin de rendre votre programme plus générique et plus puissant, réécrivez les fonctions précédentes afin qu'elles puissent créer des chaînes de Markov de n'importe quel ordre.

L'ordre de la chaîne de Markov devra être rajouté aux fonctions.

In [74]:
def compter_transitions(corpus, ordre):
    transitions = {}

    for phrase in corpus:
        prevs = ["" for _ in range(ordre)] #***
        for token in phrase:
            prev = " ".join(prevs) #***
            if prev not in transitions:
                transitions[prev] = {}
            if token not in transitions[prev]:
                transitions[prev][token] = 0
            transitions[prev][token] += 1
            prevs = prevs[1:] #***
            prevs.append(token) #***

    return transitions

In [77]:
def chaine_markov(corpus, ordre):
    transitions = compter_transitions(corpus, ordre)

    return probabilifier(transitions)

In [78]:
markov_chain = chaine_markov(corpus, 3)

In [79]:
def generer(markov_chain, ordre, start_token): #***
    maximum = NB_MOTS_MAXI
    token = start_token
    print(token, end=" ")
    prevs = ["" for _ in range(ordre)]
    prevs[-1] = token
    while token not in ponctuation and maximum > 0:
        prev = " ".join(prevs)
        nxt = sorted([x for x in markov_chain[prev].items()], key=lambda x: -x[1])
        token = nxt[0][0]
        prevs = prevs[1:]
        prevs.append(token)
        maximum -= 1
        print(token, end=" ")

In [81]:
generer(markov_chain, 3, "La")

La personne du reste qui était le plus complètement dupe de l' illusion qui m' abusait ainsi que mes parents , c' était une chose à laquelle j' avais constamment songé , une chose toute en pensées , c' était une chose à laquelle j' avais constamment songé , une chose toute en pensées , c' était une chose à laquelle j' avais constamment songé , une chose toute en pensées , c' était une chose à laquelle j' avais constamment songé , une chose toute en pensées , c' était une chose à laquelle j' avais constamment songé , une 

In [82]:
def generer(markov_chain, ordre, start_token, n_best=1):
    maximum = NB_MOTS_MAXI
    token = start_token
    print(token, end=" ")
    prevs = ["" for _ in range(ordre)]
    prevs[-1] = token
    while token not in ponctuation and maximum > 0:
        prev = " ".join(prevs)
        nxt = sorted([x for x in markov_chain[prev].items()], key=lambda x: -x[1])
        token = random.choice(nxt[:n_best])[0]
        prevs = prevs[1:]
        prevs.append(token)
        maximum -= 1
        print(token, end=" ")

In [83]:
generer(markov_chain, 3, "Si", 2)

Si j' étais monsieur votre père , je ne sais pas si c' est beaucoup ! 

In [84]:
generer(markov_chain, 3, "Si", 10)

Si les efforts de sincérité et d' émancipation de Saint-Loup ne pouvaient être trouvés que très nobles , à juger par le résultat extérieur , il était fort à craindre qu' elle se contentât de répondre de loin à ses vieux clients un clignement d' œil significatif . 

In [92]:
generer(markov_chain, 3, "Depuis", 100) 

Depuis les débuts d' Elstir , nous avons bien trouvé qu' elle marquait très mal mais nous ne savions pas qu' elle ait ses cheveux comme ça , ça donne mauvais genre . 

# Exercice 4 : générer plus qu'une seule phrase (bonus)

Dans les exercices précédents, on générait une seule phrase. Afin de compléter votre générateur, il faut à présent lui ajouter la possibilité de générer plusieurs phrases d'affilée !

## a. Se souvenir des mots de la phrase précédente

Une limitation à la génération que nous avons faite jusqu'ici est que les phrases sont vues indépendamment les unes des autres. Cela empêche la génération d'un texte plus complet qui serait constitué de plusieurs phrases.

In [86]:
generer(markov_chain, 3, "Si", 10)

Si l' on n' eût distingué deux yeux plus brillants que les autres . 

In [87]:
generer(markov_chain, 3, "Si", 10)

Si vous aimez les petites oies blanches . 